<a href="https://colab.research.google.com/github/LuizNunes2020/Laboratorios_ME315/blob/main/Lab09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install polars

In [ ]:
import polars as pl
import gdown

In [ ]:

# IDs dos arquivos
file_id_basics = "1iYqAGTtIhRLK4ycFK41hYaWK3iKfOdT2"
file_id_ratings = "1kZuj1lnXkPrNURzwYuc4FXTM7Pvknr3o"
file_id_principals = "1oLR2_mFyRHEiKlqFKt4QDVYlGH8LC4mn"

# URL do Google Drive para download
url_basics = f"https://drive.google.com/uc?id={file_id_basics}"
url_ratings = f"https://drive.google.com/uc?id={file_id_ratings}"
url_principals = f"https://drive.google.com/uc?id={file_id_principals}"

# Nome dos arquivos a serem salvos
output_basics = "title.basics0.tsv.gz"
output_ratings = "title.ratings.tsv.gz"
output_principals = "title.principals0.tsv.gz"

# Baixar os arquivos
gdown.download(url_basics, output_basics, quiet=False)
gdown.download(url_ratings, output_ratings, quiet=False)
gdown.download(url_principals, output_principals, quiet=False)


basics = pl.read_csv("title.basics0.tsv.gz", separator = "\t", null_values = "\\N",columns = ["tconst","titleType","primaryTitle","genres"])
ratings = pl.read_csv("title.ratings.tsv.gz", separator = "\t", null_values = "\\N")
principals = pl.read_csv("title.principals0.tsv.gz", separator="\t", null_values="\\N", columns=["tconst", "nconst", "category"])

Downloading...
From (original): https://drive.google.com/uc?id=1iYqAGTtIhRLK4ycFK41hYaWK3iKfOdT2
From (redirected): https://drive.google.com/uc?id=1iYqAGTtIhRLK4ycFK41hYaWK3iKfOdT2&confirm=t&uuid=fe71b713-5579-4c45-aef4-52541008ed66
To: /content/title.basics0.tsv.gz
100%|██████████| 192M/192M [00:01<00:00, 165MB/s]
Downloading...
From: https://drive.google.com/uc?id=1kZuj1lnXkPrNURzwYuc4FXTM7Pvknr3o
To: /content/title.ratings.tsv.gz
100%|██████████| 7.47M/7.47M [00:00<00:00, 89.1MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1oLR2_mFyRHEiKlqFKt4QDVYlGH8LC4mn
From (redirected): https://drive.google.com/uc?id=1oLR2_mFyRHEiKlqFKt4QDVYlGH8LC4mn&confirm=t&uuid=63c0b918-a005-4f6d-affb-49aa1648e7eb
To: /content/title.principals0.tsv.gz
100%|██████████| 671M/671M [00:07<00:00, 89.1MB/s]


'title.principals0.tsv.gz'

In [ ]:
top5 = (
  ratings.join(basics, on = "tconst")
  .sort(["averageRating", "numVotes"], descending = True)
  .select(["primaryTitle", "numVotes", "averageRating"])
  .head(5)
)
print(top5)

shape: (5, 3)
┌─────────────────────────────────┬──────────┬───────────────┐
│ primaryTitle                    ┆ numVotes ┆ averageRating │
│ ---                             ┆ ---      ┆ ---           │
│ str                             ┆ i64      ┆ f64           │
╞═════════════════════════════════╪══════════╪═══════════════╡
│ Ozymandias                      ┆ 228900   ┆ 10.0          │
│ Kadifeyi Kesfet                 ┆ 2904     ┆ 10.0          │
│ Tatilde CUK Oturur Mukemmel Du… ┆ 2878     ┆ 10.0          │
│ Inci Koleksiyonunu Kesfet!      ┆ 2812     ┆ 10.0          │
│ DeFacto Kis Festivali           ┆ 2676     ┆ 10.0          │
└─────────────────────────────────┴──────────┴───────────────┘


In [ ]:
q2 = (
    ratings.join(basics, on="tconst")
    .filter(pl.col("averageRating") > 8)
    .with_columns(pl.col("genres").str.split(",").alias("Generos"))
    .explode("Generos")
    .group_by("Generos")
    .agg(pl.count("Generos").alias("Frequencia"))
    .sort("Frequencia", descending=True)
    .head(5)
)

print(q2)


shape: (5, 2)
┌─────────────┬────────────┐
│ Generos     ┆ Frequencia │
│ ---         ┆ ---        │
│ str         ┆ u32        │
╞═════════════╪════════════╡
│ Drama       ┆ 108356     │
│ Comedy      ┆ 92208      │
│ Documentary ┆ 43531      │
│ Action      ┆ 42070      │
│ Animation   ┆ 38003      │
└─────────────┴────────────┘


In [ ]:
q3 = (
    ratings
    .join(principals, on="tconst")
    .join(basics, on="tconst")
    .filter(pl.col("averageRating") > 7.5)
    .filter(pl.col("category").is_in(["actor", "actress"]))
    .group_by("nconst")
    .agg(pl.count("primaryTitle").alias("Frequencia"))
    .sort("Frequencia", descending=True)
    .select(["nconst", "Frequencia"])
    .head(3)
)

print(q3)


shape: (3, 2)
┌───────────┬────────────┐
│ nconst    ┆ Frequencia │
│ ---       ┆ ---        │
│ str       ┆ u32        │
╞═══════════╪════════════╡
│ nm0048389 ┆ 3777       │
│ nm0217221 ┆ 2788       │
│ nm0444786 ┆ 2451       │
└───────────┴────────────┘
